In [ ]:
# !pip install splinter
# !pip install webdriver-manager
# reference blog: https://www.codecademy.com/resources/blog/web-scraping-python-beautiful-soup-mlb-stats/

In [ ]:
# import dependencies
import requests
import os
import re
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Set up Splinter Executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Visit espn salaries page
url = 'http://www.espn.com/nba/salaries'
browser.visit(url)
# Search all elements with the tag div with attribute list_text, then wait 1 second before searching components
browser.is_element_present_by_css('div.list_text', wait_time=1)

In [ ]:
# set up html parser
html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
# create the dataframe
header = soup.find('tr', class_='colhead')
columns = [col.get_text() for col in header.find_all('td')]
columns.append('season')
salaries_df = pd.DataFrame(columns = columns)
salaries_df

In [ ]:
# max_page_num = soup.find('div', class_ = 'page-numbers').text
# last_page = max_page_num[-2:]
# get a list of the years from drop down menu
menu = soup.select('option[value]')
years = [year.text for year in menu[1:]]
i = 0
# while loop to keep the loop running as long as there is an attribute to click
while True:
    # Reinitialize soup for scrapping
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # grab all the players data (RK, NAME, TEAM, SALARY) on the current page
    players = soup.find_all('tr', attrs={'class':re.compile('row')})
    # finds the year that we are currently on
    menu = soup.select('option[selected]')
    year = [menu.text for menu in menu]
    # looping through all of the soup players data
    for player in players:       
        # get info and salaries and append year
        salary = [info.get_text() for info in player.find_all('td')]
        salary.append(year[0])
        print(salary)
        
        # create a temporary df
        temp_df = pd.DataFrame(salary).transpose()
        temp_df.columns = columns
        salaries_df = pd.concat([salaries_df, temp_df], ignore_index = True)
    try:
        # find next button and press
        next_button =  browser.find_by_tag('div[class = "jcarousel-next"]')
        next_button.click()
    except:
        print(f"Done Scrapping: {year}")  
        try:
            i+=1
            print(i)
            element = browser.find_by_tag('select[class = "tablesm"]').click()
            browser.find_by_text(years[i]).click()
        except:
            print("Done with Scrapping")
            break

In [ ]:
# check df if successfully scrapped
salaries_df

In [ ]:
# quit browswer
browser.quit()

In [ ]:
# save data into csv
salaries_df.to_csv('1999-2022 Salaries.csv', index = False)